In [1]:
!pip install librosa soundfile numpy sklearn pyaudio

  Using cached PyAudio-0.2.11.tar.gz (37 kB)


  ERROR: Command errored out with exit status 1:
   command: 'C:\Users\roopa\anaconda3\python.exe' -u -c 'import sys, setuptools, tokenize; sys.argv[0] = '"'"'C:\\Users\\roopa\\AppData\\Local\\Temp\\pip-install-6eq3ver_\\pyaudio\\setup.py'"'"'; __file__='"'"'C:\\Users\\roopa\\AppData\\Local\\Temp\\pip-install-6eq3ver_\\pyaudio\\setup.py'"'"';f=getattr(tokenize, '"'"'open'"'"', open)(__file__);code=f.read().replace('"'"'\r\n'"'"', '"'"'\n'"'"');f.close();exec(compile(code, __file__, '"'"'exec'"'"'))' bdist_wheel -d 'C:\Users\roopa\AppData\Local\Temp\pip-wheel-sejpymgm'
       cwd: C:\Users\roopa\AppData\Local\Temp\pip-install-6eq3ver_\pyaudio\
  Complete output (9 lines):
  running bdist_wheel
  running build
  running build_py
  creating build
  creating build\lib.win-amd64-3.8
  copying src\pyaudio.py -> build\lib.win-amd64-3.8
  running build_ext
  building '_portaudio' extension
  error: Microsoft Visual C++ 14.0 or greater is required. Get it with "Microsoft C++ Build Tools": https

  Running setup.py clean for pyaudio
Failed to build pyaudio
    Running setup.py install for pyaudio: started
    Running setup.py install for pyaudio: finished with status 'error'


In [2]:
!pip install soundfile

In [3]:
import librosa
import soundfile
import os, glob, pickle
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import accuracy_score

In [4]:
def extract_feature(file_name, mfcc, chroma, mel):
    X, sample_rate = librosa.load(os.path.join(file_name), res_type='kaiser_fast')
    if chroma:
        stft=np.abs(librosa.stft(X))
    result=np.array([])
    if mfcc:
        mfccs=np.mean(librosa.feature.mfcc(y=X, sr=sample_rate, n_mfcc=40).T, axis=0)
        result=np.hstack((result, mfccs))
    if chroma:
        chroma=np.mean(librosa.feature.chroma_stft(S=stft, sr=sample_rate).T,axis=0)
        result=np.hstack((result, chroma))
    if mel:
        mel=np.mean(librosa.feature.melspectrogram(X, sr=sample_rate).T,axis=0)
        result=np.hstack((result, mel))
    return result

In [5]:
# Emotions in the RAVDESS & TESS dataset
emotions={
  '01':'neutral',
  '02':'calm',
  '03':'happy',
  '04':'sad',
  '05':'angry',
  '06':'fearful',
  '07':'disgust',
  '08':'surprised'
}
# Emotions to observe
observed_emotions=['neutral','calm','happy','sad','angry','fearful', 'disgust','surprised']

In [6]:
def load_data(test_size=0.2):
    x,y=[],[]
    for file in glob.glob('./NewDataSet/*_*/*.wav'):
        file_name=os.path.basename(file)
        if file_name[0]=='0':
            emotion=emotions[file_name.split("-")[2]]
        else:
            emotion=file_name.split("_")[2]
            emotion=emotion.split(".")[0]
        #print(emotion)
        if emotion not in observed_emotions:
            continue
        feature=extract_feature(file, mfcc=True, chroma=True, mel=True)
        x.append(feature)
        y.append(emotion)
    return train_test_split(np.array(x), y, test_size=test_size, train_size= 0.75,random_state=9)

In [ ]:
from tkinter import *

master = Tk()
master.geometry("500x500")
master.minsize(200,200)
master.title("Speech Emotion Detector")
frame1=Frame(master,relief=SUNKEN)
frame1.place(relx=.5, rely=.5,anchor= CENTER)
master.config(bg='#ffd6d6')
frame1.config(bg="#ffd6d6")
# f1.pack(side=TOP,fill="x")
# l.pack(padx=40)
name = Label(frame1, text ="Input",bg="#ffd6d6", font=("Arial", 25)).grid(row=0,column=0,pady= 10)
e1 = Entry(frame1,fg="blue",width=25)
e1.grid(row=1,column=0,pady= 10)
def myClick():
    file="./NewDataSet/"
    file=file+e1.get()+".wav"
    feature=extract_feature(file,mfcc=True,chroma=True,mel=True)
    #Split the dataset 
    x_train,x_test,y_train,y_test=load_data(test_size=0.25)
    # Initialize the Multi Layer Perceptron Classifier
    model=MLPClassifier(alpha=0.01, batch_size=256, epsilon=1e-08, hidden_layer_sizes=(300,), learning_rate='adaptive', max_iter=500)
    # Train the model
    model.fit(x_train,y_train)
    import time
    y_pred=model.predict(x_test)
    y_pre=model.predict([feature])
    print(y_pre[0])
    label=Label(frame1,text="Detected Emotion is: "+y_pre[0],bg="#ffd6d6",font=("Arial", 35)).grid(row=3,column=0,pady= 10)
    time.sleep(2)
    label2=Label(frame1,text="Do You want hear some songs?",bg="#ffd6d6").grid(row=4,column=0,pady= 20)
    def playsong():
        import webbrowser
        if y_pre[0]=="calm":
            webbrowser.open('https://www.youtube.com/watch?v=vyPBC25QfFM')
        elif y_pre[0]=="neutral":
            webbrowser.open('https://www.youtube.com/watch?v=o2DXt11SMNI')
        elif y_pre[0]=="happy":
            webbrowser.open('https://www.youtube.com/watch?v=MU0Yp0qmYEs')   
        elif y_pre[0]=="sad":
            webbrowser.open('https://www.youtube.com/watch?v=-xLk6Mj1xE4') 
        elif y_pre[0]=="angry":
            webbrowser.open('https://www.youtube.com/watch?v=KMhrimqyD38')  
        elif y_pre[0]=="fearful":
            webbrowser.open('https://www.youtube.com/watch?v=yfUdn8AtwSw') 
        elif y_pre[0]=="disgust":
            webbrowser.open('https://www.youtube.com/watch?v=yfUdn8AtwSw') 
        else:
            webbrowser.open('https://www.youtube.com/watch?v=cE2PmYJaj9w') 
    def dontplay():
        label3=Label(frame1,text="OK!!",bg="#ffd6d6").grid(row=7,column=1,pady= 10,padx= 5)
    btn1=Button(frame1,text="Yes",width=10,command=playsong).grid(row=5,column=0,sticky=W)
    btn2=Button(frame1,text="No",width=10,command=dontplay).grid(row=5,column=1)
b = Button(frame1,text = "Detect Emotion",width=25 ,command=myClick).grid(row=2,column=0,pady= 10)
master.mainloop()

neutral
